# Binary Classifier on Single records

### Most basic example.
This notebook will show how to set-up the features, read them from a CSV file and create a very simple feed-forward neural net to classify fraud vs. non-fraud

## Imports

In [1]:
import d373c7.features as ft
import d373c7.engines as en

## Define base feature and read the File
The base features are features found in the input file. They need to be defined after which the file can be read using the `EnginePandasNumpy`. Using the `from_csv` method

In [2]:
# Change this to read from another location
file = '../../../../data/bs140513_032310.csv'

In [3]:
age = ft.FeatureSource('age', ft.FEATURE_TYPE_CATEGORICAL)
gender = ft.FeatureSource('gender', ft.FEATURE_TYPE_CATEGORICAL)
merchant = ft.FeatureSource('merchant', ft.FEATURE_TYPE_CATEGORICAL)
category = ft.FeatureSource('category', ft.FEATURE_TYPE_CATEGORICAL)
amount = ft.FeatureSource('amount', ft.FEATURE_TYPE_FLOAT)
fraud = ft.FeatureSource('fraud', ft.FEATURE_TYPE_INT_8)

base_features = ft.TensorDefinition(
    'base', 
    [
        age,
        gender,
        merchant,
        category,
        amount,
        fraud
    ])

with en.EnginePandasNumpy() as e:
    df = e.from_csv(base_features, file)
    
df

2020-10-10 11:48:45.065 d373c7.engines.common     INFO     Start Engine...
2020-10-10 11:48:45.066 d373c7.engines.panda_numpy INFO     Pandas Version : 1.1.2
2020-10-10 11:48:45.066 d373c7.engines.panda_numpy INFO     Numpy Version : 1.19.2
2020-10-10 11:48:45.066 d373c7.engines.panda_numpy INFO     Building Panda for : base from file ../../../../data/bs140513_032310.csv
2020-10-10 11:48:45.267 d373c7.engines.panda_numpy INFO     Building Panda for : <base> from DataFrame. Inference mode <True>
2020-10-10 11:48:45.272 d373c7.engines.panda_numpy INFO     Done creating base. Shape=(594643, 6)


,age,gender,merchant,category,amount,fraud
0,4,M,M348934600,es_transportation,4.55,0
1,2,M,M348934600,es_transportation,39.68,0
2,4,F,M1823072687,es_transportation,26.89,0
3,3,M,M348934600,es_transportation,17.25,0
4,5,M,M348934600,es_transportation,35.72,0
...,...,...,...,...,...,...
594638,3,F,M1823072687,es_transportation,20.53,0
594639,4,F,M1823072687,es_transportation,50.73,0
594640,2,F,M349281107,es_fashion,22.44,0
594641,5,M,M1823072687,es_transportation,14.46,0


## Define some derived features
After we've defined the source feature, we can define some derived features. Derived features apply a form of logic to the source features. The logic depends on the type of feature. The derived features can be created using the `EnginePandasNumpy` using the `from_df` method. 

In [4]:
amount_scale = ft.FeatureNormalizeScale('amount_scale', ft.FEATURE_TYPE_FLOAT, amount)
age_oh = ft.FeatureOneHot('age_one_hot', age)
gender_oh = ft.FeatureOneHot('gender_one_hot', gender)
merchant_index = ft.FeatureIndex('merchant_index', ft.FEATURE_TYPE_INT_16, merchant)
category_index = ft.FeatureIndex('category_index', ft.FEATURE_TYPE_INT_8, category)

learning_features = ft.TensorDefinition(
    'learning', 
    [
        age_oh,
        gender_oh,
        merchant_index,
        category_index,
        amount_scale,
        fraud
    ])

In [5]:
with en.EnginePandasNumpy() as e:
    df = e.from_df(learning_features, df, False)

df

2020-10-10 11:48:50.304 d373c7.engines.common     INFO     Start Engine...
2020-10-10 11:48:50.304 d373c7.engines.panda_numpy INFO     Pandas Version : 1.1.2
2020-10-10 11:48:50.304 d373c7.engines.panda_numpy INFO     Numpy Version : 1.19.2
2020-10-10 11:48:50.304 d373c7.engines.panda_numpy INFO     Building Panda for : <learning> from DataFrame. Inference mode <False>
2020-10-10 11:48:50.329 d373c7.engines.panda_numpy INFO     Create amount_scale Normalize/Scale amount. Min. 0.00 Max. 8329.96
2020-10-10 11:48:50.343 d373c7.engines.panda_numpy INFO     Done creating learning. Shape=(594643, 16)


,age__0,age__1,age__2,age__3,age__4,age__5,age__6,age__U,gender__E,gender__F,gender__M,gender__U,merchant_index,category_index,amount_scale,fraud
0,0,0,0,0,1,0,0,0,0,0,1,0,1,1,0.000546,0
1,0,0,1,0,0,0,0,0,0,0,1,0,1,1,0.004764,0
2,0,0,0,0,1,0,0,0,0,1,0,0,2,1,0.003228,0
3,0,0,0,1,0,0,0,0,0,0,1,0,1,1,0.002071,0
4,0,0,0,0,0,1,0,0,0,0,1,0,1,1,0.004288,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
594638,0,0,0,1,0,0,0,0,0,1,0,0,2,1,0.002465,0
594639,0,0,0,0,1,0,0,0,0,1,0,0,2,1,0.006090,0
594640,0,0,1,0,0,0,0,0,0,1,0,0,15,11,0.002694,0
594641,0,0,0,0,0,1,0,0,0,0,1,0,2,1,0.001736,0
